In [1]:
import tensorflow as tf

In [2]:
train_dir = "../input/dogs-vs-cats/train"
test_dir =  "../input/dogs-vs-cats/test"

In [3]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split = 0.2)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input)

## Flow image data using flow_from_directory

In [4]:
train_images = train_gen.flow_from_directory(
    directory = train_dir,
    target_size= (224,224),
    color_mode= 'rgb',
    class_mode= 'categorical',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    subset = 'training')

val_images = train_gen.flow_from_directory(
    directory = train_dir,
    target_size= (224,224),
    color_mode= 'rgb',
    class_mode= 'categorical',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    subset = 'validation')

test_images = test_gen.flow_from_directory(
    directory = test_dir,
    target_size= (224,224),
    color_mode= 'rgb',
    class_mode= 'categorical',
    batch_size = 32,
    seed = 42,
    shuffle = False)

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [8]:
pretrained_model = tf.keras.applications.MobileNetV2(input_shape =(224,224,3), include_top=False, weights='imagenet', pooling='avg')
pretrained_model.trainable = False

In [18]:
#layers
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x1 = tf.keras.layers.Dropout(0.2)(x)
x2 = tf.keras.layers.Dense(64, activation='relu')(x1)
outputs = tf.keras.layers.Dense(2, activation ='sigmoid')(x2)



model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [19]:
history = model.fit(train_images,
                   validation_data = val_images,
                   epochs = 100,
                   callbacks = [
                       tf.keras.callbacks.EarlyStopping(
                       monitor = 'val_loss',
                       patience = 3,
                       restore_best_weights=True)
                   ])

Epoch 1/100
500/500 [==============================] - 91s 176ms/step - loss: 0.0958 - accuracy: 0.9613 - val_loss: 0.0518 - val_accuracy: 0.9803
Epoch 2/100
500/500 [==============================] - 72s 145ms/step - loss: 0.0340 - accuracy: 0.9893 - val_loss: 0.0441 - val_accuracy: 0.9825
Epoch 3/100
500/500 [==============================] - 74s 147ms/step - loss: 0.0222 - accuracy: 0.9912 - val_loss: 0.0425 - val_accuracy: 0.9877
Epoch 4/100
500/500 [==============================] - 75s 151ms/step - loss: 0.0202 - accuracy: 0.9928 - val_loss: 0.0468 - val_accuracy: 0.9862
Epoch 5/100
500/500 [==============================] - 75s 149ms/step - loss: 0.0175 - accuracy: 0.9945 - val_loss: 0.0459 - val_accuracy: 0.9885
Epoch 6/100
500/500 [==============================] - 77s 155ms/step - loss: 0.0100 - accuracy: 0.9957 - val_loss: 0.0545 - val_accuracy: 0.9850


In [20]:
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss Over Time"
)

fig.show()